In [3]:
%matplotlib ipympl

# system imports
import matplotlib
from matplotlib import pylab as plt
from scipy.optimize import leastsq
import numpy
import os

# LSST stack imports
from lsst.daf.persistence import Butler
import lsst.afw.display as afwDisplay
import lsst.afw.display.utils as afwDisplayUtils
import lsst.afw.display.utils as afwDisplayUtils
from lsst.ip.isr import IsrTask
import lsst.afw.detection as afwDetection


# Firefly client imports
from firefly_client import FireflyClient
from IPython.display import IFrame

In [4]:
isr_config = IsrTask.ConfigClass()

isr_config.doDark=False
isr_config.doFlat=False
isr_config.doFringe=False
isr_config.doDefect=False
isr_config.doAddDistortionModel=False
isr_config.doLinearize=False

In [5]:
isr = IsrTask(config=isr_config)

In [6]:
BOOTCAMP_REPO_DIR= '/project/bootcamp/repo_RTM-007/'
butler = Butler(BOOTCAMP_REPO_DIR)
print(butler.queryMetadata('raw', ['visit'], dataId={'imageType': 'FE55', 'testType': 'FE55'}))

[258334666, 258334978, 258335291, 258335602, 258335913]


In [7]:
dId = {'visit': 258334666, 'detector': 2}
raw = butler.get('raw', **dId)
bias = butler.get('bias', **dId)

In [8]:
result = isr.run(raw, bias=bias)

In [28]:
detector = result.exposure.getDetector()
amp = detector[1]

masked_img = result.exposure.getMaskedImage()
work_img = masked_img[amp.getBBox()]
arr = work_img.getImage().getArray()
    
bkgd = numpy.median(arr) # estimate a background and subtract it
arr -= bkgd

thresh = afwDetection.Threshold(100)
fs = afwDetection.FootprintSet(work_img, thresh, "BAD") # detect hits
fs = afwDetection.FootprintSet(fs, 1, False) # grow the detection footprints
fs.setMask(work_img.mask, "DETECTED")

fs.makeHeavy(work_img)

foots = fs.getFootprints()

In [29]:
len(foots)

1322

In [30]:
#afwDisplay.setDefaultBackend('matplotlib')

In [31]:
p= fs.getPeaks()[0]
p.getCentroid

AttributeError: 'lsst.afw.detection.footprintSet.FootprintSet' object has no attribute 'getPeaks'

In [33]:
# Unfortunately, afwDisplayUtils.drawFootprint does not accept an `alpha` kwarg, so we cannot control
# the transparency of the footprints being drawn
#plt.figure(figsize=(100,100))
afwDisplay.setDefaultMaskTransparency(0.5)
display = afwDisplay.Display(backend='matplotlib', reopenPlot=True)
display.scale('asinh', 'zscale')
display.setMaskTransparency(50) #  "DETECTED")
display.mtv(work_img)

with display.Buffering():
    pass
if False:
    for fs in foots:
        
        if False:
            afwDisplayUtils.drawFootprint(fs,display=display)
        else:
            display.dot('x', *fs.getPeaks()[0].getCentroid(),ctype=afwDisplay.RED)
            afwDisplayUtils.drawFootprint(fs,display=display)

#plt.show()


FigureCanvasNbAgg()

In [13]:
help(afwDisplayUtils.drawFootprint)

Help on function drawFootprint in module lsst.afw.display.utils:

drawFootprint(foot, borderWidth=0.5, origin=None, XY0=None, frame=None, ctype=None, bin=1, peaks=False, symb='+', size=0.4, ctypePeak=None, display='deferToFrame')
    Draw an afwDetection::Footprint on a display frame with the specified ctype.  Include an extra borderWidth
    pixels If origin is present, it's Added to the Footprint; if XY0 is present is Subtracted from the Footprint
    
    If peaks is True, also show the object's Peaks using the specified symbol and size and ctypePeak
    
    All Footprint coordinates are divided by bin, as is right and proper for overlaying on a binned image

